In [35]:
import os
import sys
import urllib, io

import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [6]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.path.abspath(os.path.join(os.getcwd(),'..'))
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))
jefan_dir = os.path.join(analysis_dir,'jefan')
will_dir = os.path.join(analysis_dir,'will')

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [11]:
# set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['block_construction']
coll = db['silhouette']

# which iteration name should we use?
iterationName = 'Exp2Pilot1'
# variables to check integrity of data
numTrials = 24

## Sanity Checks

In [8]:
# Ensure one to one gameID and workerId 
# Should only happen if a repeat worker gets through

query = coll.find({"$and":[
                        {'workerId':{'$exists':True}},
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {"$or":[{'iterationName':'pilot2'},
                                {'iterationName':'pilot3'},
                                {'iterationName':'pilot4'},
                                {'iterationName':'Exp2Pilot1'}]},
                        {'trialNum':0}]
                     })

df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))
#df_trial_end_full[['workerId','gameID']]


assert (np.mean(df_trial_end_full['workerId'].value_counts()) == np.mean(df_trial_end_full['gameID'].value_counts()))

### Find full datasets for Silhouette_1

In [12]:
# get ids of people with trial 23 data
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName': iterationName},
                        {'trialNum': numTrials-1}]
                     })
complete_data_df = pd.DataFrame(query)
complete_data_ids = list(complete_data_df['workerId'])

In [13]:
complete_data_ids

['A12FTSX85NQ8N9', 'A2SVNXNMD7E7EH', 'A37OUZOGQKGMW0']

In [14]:
# Filter for full datasets
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName':iterationName}]
                     })

df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))


# filter dataframe for complete datasets
df_trial_end_full_filtered = df_trial_end_full[df_trial_end_full.workerId.isin(complete_data_ids)]

# reduce to crucial information
df_trial_end_reduced_filtered = df_trial_end_full_filtered[[
                'gameID','trialNum','phase','condition',
                'eventType','score','normedScore','numBlocks',
                'timeAbsolute','timeRelative','buildTime',
                'currBonus','allVertices','nPracticeAttempts',
                'buildStartTime','buildFinishTime','targetName',
                'blockColor','blockColorID','numTargets', 'prePostSetSize',
                'numRepetitions','repetition','targetID',
                'bonusThresholdLow','bonusThresholdMid','bonusThresholdHigh',
                'timeThresholdYellow','timeThresholdRed','devMode',
                'timeBonus'
               ]]

df = df_trial_end_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])

In [32]:
out_path = os.path.join(csv_dir,'block_silhouette_{}.csv'.format(iterationName))
df.to_csv(out_path)

## Initial Block Data
Initial block placements (before physics, after snapping)

In [21]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'initial'},
                        {'iterationName':iterationName}]
                     })

df_initial_full = pd.DataFrame(list(query))

# filter dataframe for complete datasets
df_initial_full_filtered = df_initial_full[df_initial_full.workerId.isin(complete_data_ids)]

print('Loaded ' + str(df_initial_full_filtered.shape[0]) + ' complete sets of settled blocks')
# reduce to crucial information

Loaded 621 complete sets of settled blocks


In [23]:
df_initial_full_filtered.columns

Index(['F1Score', '_id', 'aID', 'allBlockDims', 'blockBodyProperties',
       'blockCenterX', 'blockCenterY', 'blockColor', 'blockColorID',
       'blockColors', 'blockDimUnits', 'blockHeight', 'blockKind', 'blockNum',
       'blockOptions', 'blockVertices', 'blockWidth', 'buildDuration',
       'canvasHeight', 'canvasWidth', 'condition', 'currBonus', 'dataType',
       'devMode', 'eventType', 'floorOptions', 'floorProperties', 'floorY',
       'gameID', 'hitID', 'incrementalScore', 'iterationName', 'menuHeight',
       'menuWidth', 'normedIncrementalScore', 'normedScore', 'nullScore',
       'numRepetitions', 'numTargets', 'numTrials', 'phase', 'points',
       'practiceDuration', 'prePostSetSize', 'prompt', 'randID',
       'relativePlacementTime', 'repetition', 'scalingFactor', 'score',
       'scoreGap', 'stimCanvasHeight', 'stimCanvasWidth', 'stimX', 'stimY',
       'stim_scale', 'targetBlocks', 'targetID', 'targetName', 'timeAbsolute',
       'timeBlockPlaced', 'timeBlockSelected

In [24]:
df_initial_reduced_filtered = df_initial_full_filtered[['gameID','trialNum','phase','condition',
                                                            'eventType', 'timeAbsolute','timeRelative',
                                                            'normedScore','currBonus','score','incrementalScore','normedIncrementalScore',
                                                            'currBonus','targetName','relativePlacementTime','iterationName',
                                                            'blockKind','targetID','blockColorID','blockColor','blockCenterX', 'blockCenterY','blockNum',
                                                           ]]

df_initial_reduced_filtered = df_initial_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])


buildstart = df[['gameID','trialNum','buildStartTime']]

df_initial_reduced_filtered = df_initial_reduced_filtered.merge(buildstart, on=['gameID', 'trialNum'], how='left')
df_initial_reduced_filtered['timePlaced'] = df_initial_reduced_filtered['timeAbsolute'] - df_initial_reduced_filtered['buildStartTime']
df_initial_reduced_filtered.loc[(df_initial_reduced_filtered.timePlaced < 0),'timePlaced'] = df_initial_reduced_filtered[df_initial_reduced_filtered.timePlaced < 0]['timeAbsolute'] - df_initial_reduced_filtered[df_initial_reduced_filtered.timePlaced < 0]['exploreStartTime']


In [28]:
df_initial_reduced_filtered[['buildStartTime','timeAbsolute','timePlaced','trialNum']]

,buildStartTime,timeAbsolute,timePlaced,trialNum
0,1.579223e+12,1.579224e+12,21281.0,0
1,1.579223e+12,1.579224e+12,35121.0,0
2,1.579223e+12,1.579224e+12,56065.0,0
3,1.579224e+12,1.579224e+12,-50718.0,1
4,1.579224e+12,1.579224e+12,-30374.0,1
5,1.579224e+12,1.579224e+12,15605.0,1
6,1.579224e+12,1.579224e+12,27650.0,1
7,1.579224e+12,1.579224e+12,35762.0,1
8,1.579224e+12,1.579224e+12,56655.0,1
9,1.579224e+12,1.579224e+12,12526.0,2


## Settled Block Data
Block data after coming to rest (after physics)

In [ ]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'settled'},
                        {'iterationName':iterationName}]
                     })

df_settled_full = pd.DataFrame(list(query))


# filter dataframe for complete datasets
df_settled_full_filtered = df_settled_full[df_settled_full.workerId.isin(complete_data_ids)]

print('Loaded ' + str(df_settled_full_filtered.shape[0]) + ' complete sets of settled blocks')
# reduce to crucial information